# [WSI] Ćwiczenie nr 3
### Algorytm MinMax

Mikołaj Wewiór 318407


In [1]:
import numpy as np
from src.board import Board


WAGES = [
    [3, 2, 3],
    [2, 4, 2],
    [3, 2, 3]
]
SIZE = 3
DEPTH = 3

In [2]:
def heurisic(state):
    score = 0
    size = state.shape[0]
    for row in range(size):
        for col in range(size):
            if state[row][col] == 'x':
                score += WAGES[row][col]
            if state[row][col] == 'o':
                score -= WAGES[row][col]
    return score

def check_win(state: np.ndarray = np.ndarray((3, 3), dtype='U7')):
    size = state.shape[0]
    for i in range(size):
        row_count = 0
        col_count = 0
        diag_up_count = 0
        diag_down_count = 0

        for j in range(size):
            if state[i][j] == state[i][0]:
                if state[i][j] != "":
                    row_count += 1
            if state[j][i] == state[0][i]:
                if state[j][i] != "":
                    col_count += 1
            if state[j][j] == state[0][0]:
                if state[j][j] != "":
                    diag_down_count += 1
            if state[-(j+1)][j] == state[0][-1]:
                if state[-(j+1)][j] != "":
                    diag_up_count += 1

        if row_count == size:
            winner = state[i][0]
            return True, winner
        if col_count == size:
            winner = state[0][i]
            return True, winner
        if diag_down_count == size:
            winner = state[0][0]
            return True, winner
        if diag_up_count == size:
            winner = state[-1][0]
            return True, winner
    return False, None


def level(state: np.ndarray):
    level = 0
    size = state.shape[0]
    for i in range(size):
        for j in range(size):
            if state[i][j] == 'o' or state[i][j] == 'x':
                level += 1
    return level

def is_board_full(state):
    return np.all(state != '')

def possible_moves(state):
    return [(i, j) for i in range(3) for j in range(3) if state[i, j] == '']


In [3]:
# def evaluate_game(state: np.ndarray) -> int:
#     score = 0
#     size = state.shape[0]
#     for row in range(size):
#         for col in range(size):
#             if state[row][col] == 'x':
#                 score += WAGES[row][col]
#             if state[row][col] == 'o':
#                 score -= WAGES[row][col]
#     return score, state

# def minmax(
#     board: Board,
#     depth: int,
#     # state: np.ndarray,
#     # action: tuple, # (int, int)
#     maximizing: bool,
#     ) -> int:
#     best_move = [None, None]
#     state = board.board
#     if board.round() == board.max_round() or depth == 0:
#         return evaluate_game(state)[0]
#     size = board.size()
#     if not maximizing: # maximizing
#         best = -24
#         for i in range(size):
#             for j in range(size):
#                 if state[i, j] != 'x' and state[i, j] != 'o':
#                     state[i, j] = board.current_player() # potencjalny bug
#                     board.board = state
#                     temp_best = best
#                     best = max(best, minmax(board, depth-1, maximizing=board.next_player())[0])
#                     if best != temp_best:
#                         best_move = [i, j]
#                     state[i, j] = ""
#         return best, best_move
#     if maximizing: # minimizing
#         best = 24
#         for i in range(size):
#             for j in range(size):
#                 # # <postać> state = np.array(tictactoe.board())
#                 if state[i, j] != 'x' and state[i, j] != 'o':
#                     state[i, j] = board.current_player() # potencjalny bug
#                     board.board = state
#                     temp_best = best
#                     best = min(best, minmax(board, depth-1, maximizing=board.next_player()))
#                     if best != temp_best:
#                         best_move = [i, j]
#                     state[i, j] = ""
#         return best, best_move
#         # best = max(best, minmax(board, depth-1, state, action=(1, 1), not maximizing))

# def alpha_pruning(
#     state: np.ndarray,
#     action: tuple, # (int, int)
#     maximizing: bool,
#     alpha: int,
#     beta: int,
#     ) -> int:
#     raise NotImplementedError


In [4]:
def evaluate_game(state: np.ndarray) -> int:
    anybody, who = check_win(state)
    if not anybody:
        return heurisic(state)
    if who == 'x':
        return 100
    if who == 'o':
        return -100

def minmax(
    state: np.ndarray,
    depth: int,
    # action: tuple, #[int, int]
    maximizing: bool,
) -> int:

    max_level = state.size
    if check_win(state)[0] or depth == 0 or level(state) == max_level:
        return evaluate_game(state)

    if maximizing:
        best_score = -1*np.inf
        for move in possible_moves(state):
            next_state = state.copy()
            next_state[move] = 'x'
            next_score = minmax(next_state, depth-1, not maximizing)
            best_score = max(best_score, next_score)
        return best_score
    if not maximizing:
        best_score = np.inf
        for move in possible_moves(state):
            next_state = state.copy()
            next_state[move] = 'o'
            next_score = minmax(next_state, depth-1, not maximizing)
            best_score = min(best_score, next_score)
        return best_score

def make_best_move(state): # , maximizing):
    best_move = None
    best_score = -1*np.inf
    for move in possible_moves(state):
        next_state = state.copy()
        next_state[move] = 'x'
        score = minmax(next_state, DEPTH, maximizing=False)  # Assuming the depth of the search as 9
        if score > best_score:
            best_score = score
            best_move = move
    return best_move

In [5]:
tictactoe = None
tictactoe = Board(3, False)
tictactoe.print()
state = np.array(tictactoe.board)
size = state.size
while tictactoe.round() < size and not tictactoe.finished():
    if not tictactoe._x_player:
        tictactoe.move()
    else:
        move = make_best_move(state)
        tictactoe.move(move)
    state = np.array(tictactoe.board)





     |     |     
  1  |  2  |  3  
_____|_____|_____
     |     |     
  4  |  5  |  6  
_____|_____|_____
     |     |     
  7  |  8  |  9  
     |     |     

-----------------------------

Current player is 'o'




     |     |     
  o  |  2  |  3  
_____|_____|_____
     |     |     
  4  |  5  |  6  
_____|_____|_____
     |     |     
  7  |  8  |  9  
     |     |     

-----------------------------


     |     |     
  o  |  2  |  3  
_____|_____|_____
     |     |     
  4  |  x  |  6  
_____|_____|_____
     |     |     
  7  |  8  |  9  
     |     |     

-----------------------------

Current player is 'o'


     |     |     
  o  |  2  |  3  
_____|_____|_____
     |     |     
  4  |  x  |  6  
_____|_____|_____
     |     |     
  7  |  8  |  o  
     |     |     

-----------------------------


     |     |     
  o  |  x  |  3  
_____|_____|_____
     |     |     
  4  |  x  |  6  
_____|_____|_____
     |     |     
  7  |  8  |  o  
     |     |     

-----------------------------

Current player is 'o'


     |     |     
  o  |  x  |  3  
_____|_____|_____
     |     |     
  4  |  x  |  6  
_____|_____|_____
     |     |     
  7  |  o  |  o  
     |     |     

---------